In [7]:
import numpy as np
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
from scipy.fft import rfft, rfftfreq #outputs half
from scipy.signal import find_peaks
from typing import Union


In [2]:
'''Column names of limbs that we care about atm:
- STRN
- LASI
- LSHO 
- LELB 
- LFIN
- LWRA
- LWRB'''

'''-------------CHOICES--------------'''
angLSHO = ["LASI","LSHO","LELB"]
angLELB = ["LSHO","LELB","LWRA"]
angLWRA = ["LELB","LWRA","LFIN"]
angRSHO = ["RASI","RSHO","RELB"]
angRELB = ["RSHO","RELB","RWRA"]
angRWRA = ["RELB","RWRA","RFIN"]

coarseness = 20 #min: 5 

#Frequency plotting things
angLST = [angLSHO,angLELB,angLWRA,angRSHO,angRELB,angRWRA] # list of angles to loop through
limbLST = ['LSHO','LELB','LWRA','RSHO','RELB','RWRA'] # list of angles to loop through

In [3]:
def np_piece_data_from_csv(num, piece, deriv=False):
    if deriv:
        filename = f'/Users/HAQbook/Desktop/graaaaphs/data/piano01_00{str(piece)}_p{str(num)}_d.csv' #performance number 1-6
    else:
        filename = f'/Users/HAQbook/Desktop/graaaaphs/data/piano01_00{str(piece)}_p{str(num)}.csv' #performance number 1-6
    repo = pd.read_csv(filename,header=0)
    columns=['Frame', 'Time (Seconds)']
    repo = repo.drop(columns, axis=1)
    vals = repo.to_numpy(dtype=float)
    mask = np.isnan(vals)
    vals[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), vals[~mask])

    return repo, vals

In [4]:
def getDegrees2D(repo, vals, POINTSx):
    '''Gets angle between three points ps=[p1,p2,p3] over all frames'''
    p1x = repo.columns.get_loc("piano_pilot_01:"+POINTSx[0]+"x") # get the x,y,z data for each of the three points over all frames
    p2x = repo.columns.get_loc("piano_pilot_01:"+POINTSx[1]+"x")
    p3x = repo.columns.get_loc("piano_pilot_01:"+POINTSx[2]+"x")

    unit_vector = lambda x: normalize(x, axis = 1, norm = 'l2') # helper function to normalise vectors, anonymous function woah

    v1 = vals[:,p1x:p1x+3] - vals[:,p2x:p2x+3] # 3 points -> 2 vectors
    v2 = vals[:,p3x:p3x+3] - vals[:,p2x:p2x+3]
    v1_u = unit_vector(v1) # vectors -> unit vectors
    v2_u = unit_vector(v2)
    ang = np.rad2deg(np.arccos(np.sum(v1_u*v2_u, axis=1))) # calculate angle between vectors
    return(ang) 

In [5]:
def standardize_len(lists: list[np.array]):
    '''standardize the length of the arrays by interpolation 
    (sort of stretching them performances to all be the same length for ease of comparison)'''
    x = np.linspace(0, len(lists[0]) - 1, len(lists[0]))
    standardized_len_lists = []
    for lst in lists:
        #Stretch/compress y to be the length of first list
        xp = np.linspace(x[0], x[-1], len(lst))
        new_lst = np.interp(x, xp, lst) 
        standardized_len_lists.append(new_lst) 
    return(standardized_len_lists)
    
def removeOutliers(y):
    # Calculate the IQR fences
    #print(f'{np.min(y)=} and {np.max(y)=}')
    x = np.linspace(np.min(y), np.max(y), num=len(y)) #set of x values that cover the range of the data and are sufficiently dense to capture any rapid changes in the y values. 
    Q1 = np.percentile(y, 25)
    Q3 = np.percentile(y, 75)
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR

    # Smooth the outliers with linear interpolation
    outliers = (y < lower_fence) | (y > upper_fence)
    x_outliers = x[outliers]
    f = interp1d(x[~outliers], y[~outliers], kind='linear', bounds_error=False, fill_value='extrapolate')
    y_smoothed = f(x_outliers)

    # Replace the outliers with the smoothed values
    y[outliers] = y_smoothed
    return(y)

def smooth(a, ker = 80):
    kernel_size = ker
    kernel = np.ones(kernel_size) / kernel_size
    a = np.convolve(a, kernel, mode='same') 
    a_cleaner = removeOutliers(a)
    return(a_cleaner)


In [8]:
def plot_func(piece: Union[int, list[int]], perf: Union[int, list[int]], limb: list[str]) -> None:
    lisANGLES, lisLABELS = [], []

    cond = isinstance(piece, list)

    if cond:
        objects = piece
        lisLABELS_str = '_perf'
        title_func = lambda x, y, z: f'{x} for performance {y} over pieces {z}'
    else:
        objects = perf
        lisLABELS_str = '_piece'
        title_func = lambda x, y, z: f'{x} for piece {y} over performances {z}'

    for obj in objects:
        if cond: repo, vals = np_piece_data_from_csv(perf, obj)
        else: repo, vals = np_piece_data_from_csv(obj, piece)
        ang = getDegrees2D(repo, vals, POINTSx = limb)
        lisANGLES.append(ang)
        lisLABELS.append(f'{limb[1]}{lisLABELS_str}{obj}')

    if cond:
        plot_degVStime2Dm(lisANGLES, lisLABELS, title = title_func(limb[1], perf, objects))
    else:
        plot_degVStime2Dm(lisANGLES, lisLABELS, title = title_func(limb[1], piece, objects))


def plot_degVStime2Dm(*data, **args):
    '''MULTI DATASOURCE!! For each array in lisANGLES, 
    plot them over length of the longest array over time.
    Label each line by the corresponding label from lisLABELS.'''
    
    lisANGLES, lisLABELS = data[0], data[1]
    lisANGLES = standardize_len(lisANGLES)
    title = args.get('title', '') #.get: If the key is not found, the default value of an empty string is used

    plt.figure(figsize=(18, 5))
    for count,ang in enumerate(lisANGLES):
        ln = np.size(lisANGLES[count],0)
        X = np.linspace(0, ln//240, ln) # x-axis for graph: frame # -> seconds 
        plt.plot(X,ang,color=((6-count)/6,0.4,(count+1)/6,0.8), label=lisLABELS[count]) # generate plot
    plt.ylabel("Degrees")
    plt.xlabel("Time (sec)")
    plt.title(title)
    plt.legend()
    plt.show() #comment out when data is needed, not plot


 Do chords have less wrist movement frequency variation than passages?